In [828]:
import pandas as pd
import numpy as np
import seaborn as sn
import tensorflow as tf
import matplotlib.pyplot as plt
from math import log
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn import neighbors, metrics

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException 
import re
from datetime import date



## Betting odds Dataframe

Source: <a href="http://tennis-data.co.uk/alldata.php" target="_blank">http://tennis-data.co.uk/alldata.php</a>

In [829]:
df_betting=pd.DataFrame()
for i in range(2002,2021):
    
    url="http://tennis-data.co.uk/%d/%d.xlsx" %(i,i)
    df=pd.read_excel(url)
    df_betting= pd.concat([df_betting,df],ignore_index=True)

KeyboardInterrupt: 

In [ ]:
#Correction for Names not respecting the syntax 
df_betting["Winner"]=df_betting["Winner"].replace({"Kwon S.W.":"Kwon S."})
df_betting["Loser"]=df_betting["Loser"].replace({"Kwon S.W.":"Kwon S."})

Create a datframe with Atp matches since 2002

In [ ]:
def checkName(Name):
    if '..' in Name:
        Name=Name.replace('..','.')
    else:
        Name=Name
        
    if '. ' in Name:
        Name=Name.replace('. ','.')
    else:
        Name=Name    
        
    length=len(Name)
    if Name[length-1]!='.':
        Name=Name+'.'
    else:
        Name=Name
    return Name

In [ ]:
df_betting_players=pd.DataFrame()
#Keep same syntax for player Name in order to have an unique ID for each player
df_betting_players["Winner"]=df_betting["Winner"].apply(lambda x: checkName(x))
df_betting_players["Loser"]=df_betting["Loser"].apply(lambda x: checkName(x))
df_betting_players["WinnerFName"]=df_betting_players["Winner"].apply(lambda x : " ".join(x.split(" ")[0:-1]))
df_betting_players["LoserFName"]=df_betting_players["Loser"].apply(lambda x : " ".join(x.split(" ")[0:-1]))

In [ ]:
df_betting_players.head()

## Players Dataframe

Source: <a href="https://www.kaggle.com/romanzdk/atp-players-overviews" target="_blank">https://www.kaggle.com/romanzdk/atp-players-overviews/</a>

In [ ]:
df_players=pd.read_csv('player_overviews.csv')
df_players=df_players.drop(df_players.columns[4:17],axis=1)
df_players=df_players.drop(df_players.columns[-1],axis=1)
df_players.columns=['AtpId','AtpName','FirstName','Name','Height','Handedness']
df_players.head()

### Change Name syntax to be similar to betting dataframe

For a future merge with the betting dataframe, we need to apply the same syntax to the Name column

In [ ]:
def PlayerName(Name,FirstName):
    length= len(Name)
    if "-" in FirstName:
        FirstName=FirstName.split("-")
        player=Name+' '+FirstName[0][0]+'.'+FirstName[1][0]+'.'
    elif " " in FirstName:
        FirstName=FirstName.split()
        if len(FirstName)>1:
            player=Name+' '+FirstName[0][0]+'.'+FirstName[1][0]+'.'
        else:
            player=Name+' '+FirstName[0]+'.'
    elif Name[length-1]=='.':
        player=Name+FirstName[0]+'.'
        
    else:
        player=Name+' '+FirstName[0]+'.'
    return player.title()

In [ ]:
df_players['NewName']=df_players.apply(lambda x: PlayerName(x["Name"],x["FirstName"]), axis=1)

Some players have their name and first letter of their firstname similar. We have to distinguished them 

### Update players dataframe

The dataset is not udpated and some current players are missing. The idea is to scrape datas from the https://www.atptour.com/ website.

In [ ]:
#Player informations' are accessible through their atp overview webpage. 
#To get this page we need the name and the id given by the website.
#A first scrape on 'https://www.atptour.com/en/rankings/singles/ gives the url of the overview page
#Then for each player, we scrape First name and Last name in their overview webpage and create a dictionnary

def AtpPlayers():
    AtpIdList=[]
    AtpNameList=[]
    FirstNameList=[]
    LastNameList=[]
    ActualRankingList=[]
    
    ##Find last ranking update date
    url='https://www.atptour.com/en/rankings/singles'
    req = Request(url, headers={'User-Agent': ''})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    update_date=soup.find(attrs={'class':'dropdown-default-label'}).get_text()
    update_date=update_date.replace('.','-').rstrip().lstrip() #clean string 

    ##Access the ranking with the first 700 players
    url='https://www.atptour.com/en/rankings/singles/?rankDate='+update_date+'&countryCode=all&rankRange=0-700'
    req = Request(url, headers={'User-Agent': ''})
    webpage = urlopen(req).read()
    soup1 = BeautifulSoup(webpage, 'html.parser')
    for i in range(0,700):
        
        ##Access the player overview and scrape their AtpId, AtpName, FirstName and Name
        
        url_overview='https://www.atptour.com' + soup1.select('.player-cell a')[i]['href']
        #AtpName and AtpId are present in the href
        AtpName=url_overview.split("/")[5]
        AtpId=url_overview.split("/")[6]
        
        req = Request(url_overview, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup2 = BeautifulSoup(webpage, 'html.parser')
        try:
            FirstName = soup2.find( attrs={'class': 'first-name'}).get_text()
        except AttributeError:
            FirstName=np.nan
        
        try:
            Name = soup2.find( attrs={'class': 'last-name'}).get_text()
        except AttributeError:
            Name=np.nan
            
        
            
        AtpIdList.append(AtpId)
        AtpNameList.append(AtpName)
        FirstNameList.append(FirstName)
        LastNameList.append(Name)
        ActualRankingList.append(i+1) #create an Actual ranking list useful for validation process
        
        Dictionnary={'AtpId':AtpIdList,'AtpName':AtpNameList,'FirstName':FirstNameList,'Name':LastNameList,'ActualRanking':ActualRankingList}

    return Dictionnary

In [ ]:
df_AtpPlayers=AtpPlayers() #get a dictionnary with scraped datas
df_AtpPlayers=pd.DataFrame(df_AtpPlayers) #convert dictionnary into dataframe
df_AtpPlayers['NewName']=df_AtpPlayers.apply(lambda x: PlayerName(x["Name"],x["FirstName"]), axis=1)
#df_AtpPlayers=df_AtpPlayers.drop(["FirstName"],axis=1) #column not needed anymore

In [ ]:
df_total_players=pd.concat([df_players, df_AtpPlayers], ignore_index=True) # concat the scraping dataframe with the original one
df_total_players.drop_duplicates(subset ="AtpId",keep = "last", inplace = True) # KEEP UNIQUE VALUES
print(df_total_players.shape)
df_total_players.head()

In [ ]:
#df_total_players=df_total_players.sort_values(["Name","FirstName"]).reset_index(drop=True)

In [ ]:
#df_total_players["Name"]=duplicatedNames(df_total_players)

In [ ]:
#df_total_players=df_total_players.drop(["NewName"],axis=1) #column not needed anymore
#df_total_players.head()

In [ ]:
print(df_total_players.shape)

### Select only commun players

In [ ]:
df_selected_players=df_total_players[df_total_players["Name"].isin(df_betting_players["WinnerFName"])|df_total_players["Name"].isin(df_betting_players["LoserFName"])].reset_index(drop=True) 

In [ ]:
df_selected_players.head()

In [ ]:
def SimilarNames(df):
    column=[df.loc[0,"NewName"]]
    for i in range(1,len(df)):
        
        FirstName=df.loc[i,"FirstName"]
        Name=df.loc[i,"Name"]
        
        j=0
        while df.loc[i,"NewName"]in column:
            
            df.loc[i,"NewName"]=Name+' '+FirstName[0:j+1]+'.'
            j+=1
        
        
        column.append(df.loc[i,"NewName"])
    return column

In [ ]:
def UniquePlayers(df_selected_players, df_betting_players):
    
    ##########################################################################################
    ## With this function we want to deal with Players which have the same "NewName".       ##
    ## Some players in the df_betting_players are unique but not in the df_selected_players.##
    ## In this case, the idea is to keep only the player with an actual ranking in order to ##
    ## have a logical further merge. If there is no actual players we keep them all.        ##
    ## The second step will be to use the SimilarNames function to differentiate players    ##
    ## with the same Name in df_selected_players                                            ##
    ##########################################################################################

    newdf=[]
    for i in range(0,len(df_selected_players)):
        
        occurrence=df_selected_players[df_selected_players["NewName"].str.contains(df_selected_players.iloc[i]["NewName"])] # number of occurrence of the NewName in the dataframe
        
        #find unique players in the betting dataframe containing the "NewName" value
        betting_unique=df_betting_players['Winner'][df_betting_players["Winner"].str.contains(df_selected_players.iloc[i]["NewName"])]
        betting_unique=betting_unique.append(df_betting_players['Loser'][df_betting_players["Loser"].str.contains(df_selected_players.iloc[i]["NewName"])])
        betting_unique=betting_unique.unique()
                                             
        if (len(betting_unique)==1) & (occurrence.shape[0]!=1): #if there is a unique player in df_selected_players_betting_payers for multiple players in df_selected_players
            currentPlayers= occurrence[occurrence['ActualRanking'].notna()] #select rows with an 'ActualRanking' value
            if len(currentPlayers)!=0: 
                for j in currentPlayers.index:
                    newdf.append(df_selected_players.iloc[j]) #add only rows with an ActualRanking value
            else:
                newdf.append(df_selected_players.iloc[i])
        else:
            newdf.append(df_selected_players.iloc[i])
            
    newdf=pd.DataFrame(newdf) # convert to pandas dataframe
    newdf.drop_duplicates(subset ="AtpId",keep = "last", inplace = True) # drop duplicated Atp players
    newdf=newdf.sort_values(["Name","FirstName"]).reset_index(drop=True) # sort players by their Name and firstname
    newdf["NewName"]=SimilarNames(newdf) # apply SimilarNames function
    
    return newdf

In [ ]:
df_selected_players=UniquePlayers(df_selected_players,df_betting_players)

In [845]:
df_selected_players[df_selected_players["NewName"].str.contains("Andujar")]

,AtpId,AtpName,FirstName,Name,Height,Handedness,NewName,ActualRanking,Id
62,a596,pablo-andujar,Pablo,Andujar,180.0,Right-Handed,Andujar P.,80.0,62


In [846]:
df_players[df_players["NewName"].str.contains("Andujar")]

,AtpId,AtpName,FirstName,Name,Height,Handedness,NewName
155,a596,pablo-andujar,Pablo,Andujar,180.0,Right-Handed,Andujar P.


In [ ]:
print(df_selected_players.shape)

Height, Handedness and Backhand are interesting criterias to analyse but we saw in the model prediction notebook that backhand had too many 'Nan' values even after scraping. Thanks to the AtId and AtpNAme we scraped on the website https://www.atptour.com/ 

In [ ]:
def scraping(atpId,atpName,height,handedness):
    
    try:
        url_atp = 'https://www.atptour.com/en/players/'+atpName+'/'+atpId+'/overview'
        req = Request(url_atp, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, 'html.parser')

        if ((height!=height) or (height==0.0)): #check NaN value or height equals to 0
            try:
                height = soup.find( attrs={'class': 'table-height-cm-wrapper'}).get_text()
                height= height.replace('(','').replace(')','').replace('cm','')
                if str(height)=='0':
                    height=np.nan
                else:
                    height=height
            except AttributeError:
                height=np.nan
        else:
            height=height

        if handedness!=handedness: #check NaN value
            try:
                plays= soup.find_all( attrs={'class':"table-value"})
                if len(plays)>=2:
                    plays=plays[2].get_text()
                    if len(plays)>1:
                        plays=plays.split(", ")

                        handedness=plays[0].replace("\r","")
                        handedness=handedness.replace("\n","")  
                    else:
                        handedness=plays.replace("\r","")
                        handedness=handedness.replace("\n","")
                    if handedness=='':
                        handedness=np.nan
                    else:
                        handedness=handedness
                else:
                    handedness=np.nan
            except AttributeError:
                handedness=np.nan
        else:
            handedness=handedness

        
    except:
        height=handedness=np.nan
    
    
    
    return height,handedness

In [ ]:
def UpdateRanking(atpId,atpName):
    url_atp = 'https://www.atptour.com/en/players/'+atpName+'/'+atpId+'/overview'
    req = Request(url_atp, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    soup = BeautifulSoup(webpage, 'html.parser')
    try:
        Ranking = soup.find( attrs={'class': 'data-number'}).get_text()
    except:
        Ranking=np.nan
    try:
        int(Ranking)
        return Ranking
    except ValueError: #For inactive players 
        return np.nan

In [ ]:
df_selected_players.dtypes

In [ ]:
df_selected_players[['Height','Handedness']]=df_selected_players.apply(lambda x : scraping(x['AtpId'],x['AtpName'],x['Height'],x['Handedness']),axis=1, result_type="expand")
df_selected_players.head()

In [ ]:
df_selected_players["Height"] = df_selected_players.Height.astype(float) # convert to float

In [ ]:
df_selected_players['Id']=df_selected_players.index

In [ ]:
df_copy=df_selected_players.copy()

In [ ]:
df_selected_players.head()

## Model Dataframe

### Clean Dataframe

We can notice that depending the year, Betting Broker are not the same. So it's better to use the Min/Max and Average of betting odds. 

In [ ]:
#Fill NaN values
df_betting['MaxW']=df_betting['MaxW'].fillna(round(df_betting[['B365W', 'CBW', 'GBW', 'IWW','SBW', 'B&WW', 'EXW', 'PSW', 'UBW', 'LBW', 'SJW']].max(axis=1),2))
df_betting['MaxL']=df_betting['MaxL'].fillna(round(df_betting[['B365L', 'CBL', 'GBL', 'IWL','SBL', 'B&WL', 'EXL', 'PSL', 'UBL', 'LBL', 'SJL']].max(axis=1),2))
df_betting['AvgW']=df_betting['AvgW'].fillna(round(df_betting[['B365W', 'CBW', 'GBW', 'IWW','SBW', 'B&WW', 'EXW', 'PSW', 'UBW', 'LBW', 'SJW']].mean(axis=1),2))
df_betting['AvgL']=df_betting['AvgL'].fillna(round(df_betting[['B365L', 'CBL', 'GBL', 'IWL','SBL', 'B&WL', 'EXL', 'PSL', 'UBL', 'LBL', 'SJL']].mean(axis=1),2))

In [ ]:
# drop useless datas
df_betting=df_betting[df_betting['MaxW'].notna() & df_betting['MaxL'].notna()] # select rows where betting odds are present
df_betting=df_betting.drop(df_betting[df_betting["Comment"]!="Completed"].index) #keep only completed matches
df_betting=df_betting.drop(['Comment','Date','ATP','Best of'],axis=1) # drop useless columns
df_betting=df_betting.drop(['B365W', 'CBW', 'GBW', 'IWW','SBW', 'B&WW', 'EXW', 'PSW', 'UBW', 'LBW', 'SJW','B365L', 'CBL', 'GBL', 'IWL','SBL', 'B&WL', 'EXL', 'PSL', 'UBL', 'LBL', 'SJL'],axis=1)
df_betting=df_betting.drop(['W1','L1','W2','L2','W3','L3','W4','L4','W5','L5','Wsets','Lsets','WPts','LPts'],axis=1)
df_betting=df_betting.drop(df_betting[(df_betting["WRank"]=='NR') |  (df_betting["LRank"]=='NR')].index) # drop matches with No ranked players
df_betting=df_betting.dropna()
df_betting["LRank"] = df_betting.LRank.astype(float)# no more 'NR' players so we can convert into float

### Preprocessing data

In [ ]:
#Keep same syntax for player Name in order to have an unique ID for each player
df_betting["Winner"]=df_betting["Winner"].apply(lambda x: checkName(x))
df_betting["Loser"]=df_betting["Loser"].apply(lambda x: checkName(x))

In [ ]:
#Some Series have been renamed over years
df_betting['Series'].replace({"International Gold":"ATP500","Masters Cup":"ATP Finals","Masters":"ATP Finals"},inplace =True)
#Delete ended white space
df_betting['Location']=df_betting['Location'].apply(lambda x : x.rstrip())

In [ ]:
#Rename columns: Winner -> Player1 & Loser -> Player2
df_betting.rename(columns={'Winner':'Player1','Loser':'Player2','WRank':'P1Rank','LRank':'P2Rank','MaxW':'MaxP1','MaxL':'MaxP2','AvgW':'AvgP1','AvgL':'AvgP2'}, inplace=True)
#Create the column for the label prediction: Player1 Win
df_betting['P1Winner']=1 

In the actual format all winners are Player 1, it will be a problem for the model.
So for random rows, Players 1 and Player 2 datas are switched

In [ ]:
mask=df_betting.sample(frac=.5).index
df_betting.loc[mask,['Player1','Player2','P1Rank','P2Rank','MaxP1','MaxP2','AvgP1','AvgP2']] = df_betting.loc[mask,['Player2','Player1','P2Rank','P1Rank','MaxP2','MaxP1','AvgP2','AvgP1']].values
df_betting.loc[mask,['P1Winner']]=0

In [ ]:
df_betting=df_betting.merge(df_selected_players,left_on='Player1',right_on="NewName", how='left',suffixes=['P2','P1']) #first merge for winner player
df_betting=df_betting.merge(df_selected_players,left_on='Player2',right_on="NewName", how='left',suffixes=['P1','P2'])#second merge for loser player

In [ ]:
df_betting=df_betting.drop(['MaxP1','MaxP2','NewNameP1','NewNameP2','Tournament','Round','FirstNameP1','FirstNameP2','NameP1','NameP2',"AtpIdP1","AtpIdP2","ActualRankingP1","ActualRankingP2","AtpNameP1","AtpNameP2","Player1","Player2"],axis=1) # keep only the player ID
df_betting=df_betting.dropna() # delete rows with NaN values
df_betting=df_betting.drop_duplicates() #delete duplicated rows

In [ ]:
df_betting["HeightP1"] = df_betting.HeightP1.astype(float) # convert to float
df_betting["HeightP2"] = df_betting.HeightP2.astype(float) # convert to float

In [ ]:
#move column P1Winner at the end 
df_betting=df_betting[['Location', 'Series', 'Court', 'Surface', 'P1Rank', 'P2Rank', 'AvgP1', 'AvgP2','HeightP1', 'HandednessP1', 'IdP1', 'HeightP2', 'HandednessP2', 'IdP2','P1Winner']]
df_betting.head()

Columns label encoding

In [ ]:
label_dictionnary=[] 

In [ ]:
def Labelizer(column):
    label=sorted(column.unique())
    dictionnary = { val : idx for idx,val in enumerate(label) }
    label_dictionnary.append(dictionnary) #save label encoder for validation test
    colLab=column.map(dictionnary)
    return colLab

In [ ]:
df_betting=df_betting.apply(lambda x: Labelizer(x) if x.dtype==object else x,axis=0)

In [ ]:
df_betting.head()

In [ ]:
print(label_dictionnary)

## Model

### Multilayers perceptron

In [ ]:
X=df_betting.drop(['P1Winner'],axis=1)
y=df_betting['P1Winner']


Ml_model=tf.keras.Sequential([
    tf.keras.layers.Dense(16,input_dim=X.shape[1],activation="relu"),
    #tf.keras.layers.Dense(12,activation="relu"),
    tf.keras.layers.Dense(8,activation="relu"),
    #tf.keras.layers.Dense(4,activation="relu"),
    tf.keras.layers.Dense(1,activation="sigmoid"),
])

#Compile the model

Ml_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Train the model

history=Ml_model.fit(X,y,epochs=200, batch_size=250,verbose=0)

## Validation 

The idea is to scrape https://www.oddsportal.com/matches/tennis/ and find ATP matches of the day.

In [ ]:
def PlayerSyntax(playerName):
    
    playerName=checkName(playerName)#first syntax check for dots
    
    char_positions=[pos for pos, char in enumerate(playerName) if char == "-"] #array of character position in string
    if char_positions: ##  if not empty array
        for i in char_positions:
            if playerName[i-1].isupper(): #check if the character before is an uppercase
                #convert to list in order to change specific character 
                new=list(playerName)
                new[i]="."
                playerName="".join(new)
                
    if playerName == "Kwon Soonwoo":
        playerName="Kwon S."
    elif playerName == "Ramos A.":
        playerName="Ramos-Vinolas A."
    #elif playerName == "Andujar-Alba P.":
    #    playerName="Andujar P."
    #elif playerName == "Carreno-Busta P.":
    #    playerName="Carreno Busta P."
        

    return playerName    


In [ ]:
def DayMatches():
    #############################################################################
    ## Scrape Location, Players, Odds and Winner of each ATP matches of the day ##
    #############################################################################
    url_odds="https://www.oddsportal.com/matches/tennis/"

    driver = webdriver.Chrome(executable_path=r'/Users/pierremecchia/Desktop/chromedriver')
    driver.get(url_odds)
    sleep(2)

    table=driver.find_elements_by_xpath("//*[@id='table-matches']/table/tbody/tr[contains(@class,'dark')]")

    list_href=[]
    list_location=[]
    list_player1=[]
    list_player2=[]
    list_odd1=[]
    list_odd2=[]
    list_winnerP1=[]    

    ## find all ATP tournament and keep their link
    for row in table:
        if ("ATP" in row.text) and (not "Doubles" in row.text): #only interested in single atp matches
            href=row.find_element_by_xpath('./th[1]/a[2]').get_attribute('href')
            if href not in list_href:
                list_href.append(href)
    driver.quit()
    

    for href in list_href:

        driver = webdriver.Chrome(executable_path=r'/Users/pierremecchia/Desktop/chromedriver')
        driver.get(href)
        sleep(2)
        ##find location of the atp tournament
        h1=driver.find_element_by_xpath('//*[@id="col-content"]/h1').text
        
        #clean location string
        location=h1.replace('Betting Odds','')
        location=location.replace('ATP ','')
        location=re.sub(r'\([^)]*\)', '', location)#delete special characters 
        location=location.rstrip().lstrip() #delete white space at the beginning and end of string
        
        matches=driver.find_elements_by_xpath("//*[@id='tournamentTable']/tbody/tr") #List of matches in the tournament
        doScrap=False
        for match in matches:
            try:
                className=match.get_attribute("class")
                if className=='center nob-border': #create a counter in order to scrape only matches of the day
                    date=match.find_element_by_xpath('./th[1]/span').text
                    if 'Today' in date:
                        doScrap=True
                    else:
                        doScrap=False
                if doScrap==True:
                    if not('dark' or 'nob-border' or 'table-dummyrow')in className:

                        try:
                            if 'deactivate' in className: #for matches completed or currently playing (live matches)
                                players=match.find_element_by_xpath('./td[2]/a').text

                                if players=="": #append for live matches
                                    players=match.find_element_by_xpath('./td[2]/a[2]').text

                                #find odds
                                odd1=match.find_element_by_xpath('./td[4]/a').text
                                odd1_className=match.find_element_by_xpath('./td[4]').get_attribute("class")

                                odd2=match.find_element_by_xpath('./td[5]/a').text
                                odd2_className=match.find_element_by_xpath('./td[5]').get_attribute("class")
                                
                                #find winner of the match
                                winnerP1=match.find_element_by_xpath('./td[3]').text

                                if ("ret" in winnerP1) or ("canc" in winnerP1): #no winner for canceled or retirement matches
                                    winnerP1=np.nan
                                elif (("result-ok" not in odd1_className) & ("result-ok" not in odd2_className)): #match not finished
                                    winnerP1=np.nan
                                elif int(winnerP1[0])<int(winnerP1[2]):
                                    winnerP1=0
                                else:
                                    winnerP1=1
                            else: # for matches not played yet
                                players=match.find_element_by_xpath('./td[2]/a[not(contains(@class,"ico-tv-tournament"))]').text
                                odd1=match.find_element_by_xpath('./td[3]/a').text
                                odd2=match.find_element_by_xpath('./td[4]/a').text
                                winnerP1=np.nan
                            
                            #Clean players names
                            player1=players.split(" - ")[0]
                            player1=PlayerSyntax(player1)
                            player2=players.split(" - ")[1]
                            player2=PlayerSyntax(player2)
                            
                        
                            list_location.append(location)
                            list_player1.append(player1)
                            list_player2.append(player2)
                            list_odd1.append(odd1)
                            list_odd2.append(odd2)
                            list_winnerP1.append(winnerP1)
                            
                        except :
                            continue

            except NoSuchElementException:
                continue
                
        driver.quit()
            
    dictionnary={"Location":list_location,"Player1":list_player1,"Player2":list_player2,"AvgP1":list_odd1,"AvgP2":list_odd2,"P1Winner":list_winnerP1} #create a dictionnary
    df_matchs=pd.DataFrame(dictionnary) #convert dictionnary into pandas dataframe
    
    return df_matchs
            
    

In [ ]:
def TournamentsData(locations_list):
    
    ##################################################################################
    ## Scrape Series, Surface and Court of tournaments present in the locations_list ##
    ##################################################################################
    
    
    url_tourn="https://www.atptour.com/en/tournaments/"

    driver = webdriver.Chrome(executable_path=r'/Users/pierremecchia/Desktop/chromedriver')
    driver.get(url_tourn)
    sleep(4)
    
    #list all tournaments in the current month
    month_tournament=driver.find_elements_by_xpath("//*[@id='contentAccordionWrapper']/div[contains(@class,'expand')][1]/div[2]/div/table/tbody/tr")

    list_series=[]
    list_court=[]
    list_location=[]
    list_surface=[]
    
    for tournament in month_tournament:
        try:
            #scrape location for each tournament in the list
            location=tournament.find_element_by_xpath('./td[2]/span[1]').text 
            town=location.split(",")[0]
            
            if town in locations_list: #identify tournaments that we have to scrape
                try:
                    #define the serie of the tournament thanks to the image source
                    series=tournament.find_element_by_xpath('./td[1]/img').get_attribute("src")
                    if "250.png" in series:
                        series="ATP250"
                    elif "500.png" in series:
                        series="ATP500"
                    elif "1000.png" in series:
                        series="Masters 1000"
                    elif "grandslam.png" in series:
                        series="Grand Slam"
                    elif "finals.svg" in series:
                        series="ATP Finals"
                    else:
                        series=np.nan
                except:
                    series=np.nan
                try:
                    #scrape court and surface of the tournament
                    playground=tournament.find_element_by_xpath('./td[3]/table/tbody/tr/td[2]/div/div').text
                    playground=playground.split(" ")
                    court=playground[0]
                    surface=playground[1]
                    
                except:
                    court=np.nan
                    surface=np.nan
                    
                    
                list_series.append(series)
                list_surface.append(surface)
                list_court.append(court)
                list_location.append(town)
        except:
            continue
    driver.quit()

    loc_dict={"Location":list_location,"Series":list_series,"Court":list_court,"Surface":list_surface} #create a dictionnary
    df_tournaments=pd.DataFrame(loc_dict) # convert dictionnary into pandas dataframe
    
    return df_tournaments

In [848]:
df_matchs=DayMatches() #Scrape atp matches of the day
locations_list=df_matchs['Location'].unique() #location list of tournaments of the day
df_tournaments=TournamentsData(locations_list) # scrape additionnals datas of the tournaments of the day

In [849]:
df_tournaments.head()

,Location,Series,Court,Surface
0,Barcelona,ATP500,Outdoor,Clay
1,Belgrade,ATP250,Outdoor,Clay


In [850]:
df_validation=df_matchs.merge(df_tournaments,on="Location")
df_validation.head(20)

,Location,Player1,Player2,AvgP1,AvgP2,P1Winner,Series,Court,Surface
0,Barcelona,Chardy J.,Basilashvili N.,1.43,2.85,NaN,ATP500,Outdoor,Clay
1,Barcelona,Simon G.,Andujar-Alba P.,2.53,1.52,NaN,ATP500,Outdoor,Clay
2,Barcelona,Tsonga J.W.,Gerasimov E.,2.83,1.43,NaN,ATP500,Outdoor,Clay
3,Barcelona,Zapata Miralles B.,Kuznetsov An.,1.58,2.38,NaN,ATP500,Outdoor,Clay
4,Barcelona,Nagal S.,Herbert P.,2.00,1.81,NaN,ATP500,Outdoor,Clay
5,Barcelona,Nishikori K.,Pella G.,1.23,4.18,NaN,ATP500,Outdoor,Clay
6,Barcelona,Paire B.,Gaio F.,1.75,2.08,NaN,ATP500,Outdoor,Clay
7,Barcelona,Ivashka I.,Griekspoor T.,1.45,2.75,NaN,ATP500,Outdoor,Clay
8,Barcelona,Monteiro T.,Munar J.,3.28,1.34,NaN,ATP500,Outdoor,Clay
9,Barcelona,Norrie C.,Caruso S.,2.25,1.65,NaN,ATP500,Outdoor,Clay


In [851]:
def PlayerNotFound(Player,df_selected_players):
    if Player not in (df_selected_players["NewName"]):
        firstName=Player.split(" ")[-1]
        Name=" ".join(Player.split(" ")[0:-1])

        if df_selected_players["NewName"].str.contains(Name+" "+firstName[0]).any():
            similarPlayer=[]
            i=0
            while len(similarPlayer) != 1:
                similarPlayer=df_selected_players["NewName"][df_selected_players["NewName"].str.contains(Name+" "+firstName[0:i+1])].reset_index(drop=True)
                i+=1
                if i==len(firstName):
                    similarPlayer=Player
                    break
            Player=similarPlayer
        elif df_selected_players["NewName"].str.contains(Name.replace("-"," ")+" "+firstName[0]).any():
            similarPlayer=[]
            i=0
            while len(similarPlayer) != 1:
                similarPlayer=df_selected_players["NewName"][df_selected_players["NewName"].str.contains(Name.replace("-"," ")+" "+firstName[0:i+1])].reset_index(drop=True)
                i+=1
                if i==len(firstName):
                    similarPlayer=Player
                    break
            Player=similarPlayer
        elif df_selected_players["NewName"].str.contains(Name.split("-")[0]+" "+firstName[0]).any():
            similarPlayer=[]
            i=0
            while len(similarPlayer) != 1:
                similarPlayer=df_selected_players["NewName"][df_selected_players["NewName"].str.contains(Name.split("-")[0]+" "+firstName[0:i+1])].reset_index(drop=True)
                i+=1
                if i==len(firstName):
                    similarPlayer=Player
                    break
            Player=similarPlayer
        else:
            Player=Player
        
    return Player

In [852]:
df_validation['Player1']=df_validation.apply(lambda x: PlayerNotFound(x["Player1"],df_selected_players),axis=1)
df_validation['Player2']=df_validation.apply(lambda x: PlayerNotFound(x["Player2"],df_selected_players),axis=1)
df_validation.head(20)

,Location,Player1,Player2,AvgP1,AvgP2,P1Winner,Series,Court,Surface
0,Barcelona,Chardy J.,Basilashvili N.,1.43,2.85,NaN,ATP500,Outdoor,Clay
1,Barcelona,Simon G.,Andujar P.,2.53,1.52,NaN,ATP500,Outdoor,Clay
2,Barcelona,Tsonga J.W.,Gerasimov E.,2.83,1.43,NaN,ATP500,Outdoor,Clay
3,Barcelona,Zapata Miralles B.,Kuznetsov An.,1.58,2.38,NaN,ATP500,Outdoor,Clay
4,Barcelona,Nagal S.,Herbert P.H.,2.00,1.81,NaN,ATP500,Outdoor,Clay
5,Barcelona,Nishikori K.,Pella G.,1.23,4.18,NaN,ATP500,Outdoor,Clay
6,Barcelona,Paire B.,Gaio F.,1.75,2.08,NaN,ATP500,Outdoor,Clay
7,Barcelona,Ivashka I.,Griekspoor T.,1.45,2.75,NaN,ATP500,Outdoor,Clay
8,Barcelona,Monteiro T.,Munar J.,3.28,1.34,NaN,ATP500,Outdoor,Clay
9,Barcelona,Norrie C.,Caruso S.,2.25,1.65,NaN,ATP500,Outdoor,Clay


In [853]:
df_validation=df_validation.merge(df_selected_players,left_on='Player1',right_on="NewName", how='left',suffixes=['P2','P1']) #first merge for winner player
df_validation=df_validation.merge(df_selected_players,left_on='Player2',right_on="NewName", how='left',suffixes=['P1','P2'])#second merge for loser player

In [854]:
df_validation=df_validation.drop(['FirstNameP1','FirstNameP2','NameP1','NameP2',"AtpIdP1","AtpIdP2","AtpNameP1","AtpNameP2"],axis=1) # drop useless features
df_validation=df_validation[['Location','Player1','Player2','Series', 'Court', 'Surface', 'ActualRankingP1', 'ActualRankingP2', 'AvgP1', 'AvgP2','HeightP1', 'HandednessP1', 'NewNameP1', 'IdP1', 'HeightP2', 'HandednessP2', 'NewNameP2', 'IdP2','P1Winner']]#ordering columns

In [855]:
df_validation.head(20)

,Location,Player1,Player2,Series,Court,Surface,ActualRankingP1,ActualRankingP2,AvgP1,AvgP2,HeightP1,HandednessP1,NewNameP1,IdP1,HeightP2,HandednessP2,NewNameP2,IdP2,P1Winner
0,Barcelona,Chardy J.,Basilashvili N.,ATP500,Outdoor,Clay,53.0,36.0,1.43,2.85,188.0,Right-Handed,Chardy J.,306,185.0,Right-Handed,Basilashvili N.,126.0,NaN
1,Barcelona,Simon G.,Andujar P.,ATP500,Outdoor,Clay,68.0,80.0,2.53,1.52,183.0,Right-Handed,Simon G.,1563,180.0,Right-Handed,Andujar P.,62.0,NaN
2,Barcelona,Tsonga J.W.,Gerasimov E.,ATP500,Outdoor,Clay,72.0,78.0,2.83,1.43,188.0,Right-Handed,Tsonga J.W.,1740,196.0,Right-Handed,Gerasimov E.,594.0,NaN
3,Barcelona,Zapata Miralles B.,Kuznetsov An.,ATP500,Outdoor,Clay,146.0,481.0,1.58,2.38,183.0,Right-Handed,Zapata Miralles B.,1913,185.0,Right-Handed,Kuznetsov An.,964.0,NaN
4,Barcelona,Nagal S.,Herbert P.H.,ATP500,Outdoor,Clay,135.0,70.0,2.00,1.81,178.0,Right-Handed,Nagal S.,1213,188.0,Right-Handed,Herbert P.H.,725.0,NaN
5,Barcelona,Nishikori K.,Pella G.,ATP500,Outdoor,Clay,39.0,48.0,1.23,4.18,178.0,Right-Handed,Nishikori K.,1241,183.0,Left-Handed,Pella G.,1304.0,NaN
6,Barcelona,Paire B.,Gaio F.,ATP500,Outdoor,Clay,35.0,136.0,1.75,2.08,196.0,Right-Handed,Paire B.,1284,178.0,Right-Handed,Gaio F.,559.0,NaN
7,Barcelona,Ivashka I.,Griekspoor T.,ATP500,Outdoor,Clay,111.0,143.0,1.45,2.75,193.0,Right-Handed,Ivashka I.,776,188.0,Right-Handed,Griekspoor T.,659.0,NaN
8,Barcelona,Monteiro T.,Munar J.,ATP500,Outdoor,Clay,77.0,81.0,3.28,1.34,183.0,Left-Handed,Monteiro T.,1169,183.0,Right-Handed,Munar J.,1198.0,NaN
9,Barcelona,Norrie C.,Caruso S.,ATP500,Outdoor,Clay,52.0,89.0,2.25,1.65,188.0,Left-Handed,Norrie C.,1247,185.0,Right-Handed,Caruso S.,287.0,NaN


In [856]:
df_validation=df_validation.drop(["Player1","Player2","NewNameP1","NewNameP2"],axis=1) # features already present in idP1 and idP2

In [857]:
df_validation['AvgP1']=df_validation.AvgP1.astype(float)#convert column to float
df_validation['AvgP2']=df_validation.AvgP2.astype(float)#convert column to float

In [873]:
df_val_test=df_validation.copy()

In [874]:
df_val_test=df_val_test[df_val_test['IdP1'].notna()&df_val_test['IdP2'].notna()] #keep rows without NaN values in Id columns
df_val_test.head(15)

,Location,Series,Court,Surface,ActualRankingP1,ActualRankingP2,AvgP1,AvgP2,HeightP1,HandednessP1,IdP1,HeightP2,HandednessP2,IdP2,P1Winner
0,Barcelona,ATP500,Outdoor,Clay,53.0,36.0,1.43,2.85,188.0,Right-Handed,306,185.0,Right-Handed,126.0,NaN
1,Barcelona,ATP500,Outdoor,Clay,68.0,80.0,2.53,1.52,183.0,Right-Handed,1563,180.0,Right-Handed,62.0,NaN
2,Barcelona,ATP500,Outdoor,Clay,72.0,78.0,2.83,1.43,188.0,Right-Handed,1740,196.0,Right-Handed,594.0,NaN
3,Barcelona,ATP500,Outdoor,Clay,146.0,481.0,1.58,2.38,183.0,Right-Handed,1913,185.0,Right-Handed,964.0,NaN
4,Barcelona,ATP500,Outdoor,Clay,135.0,70.0,2.00,1.81,178.0,Right-Handed,1213,188.0,Right-Handed,725.0,NaN
5,Barcelona,ATP500,Outdoor,Clay,39.0,48.0,1.23,4.18,178.0,Right-Handed,1241,183.0,Left-Handed,1304.0,NaN
6,Barcelona,ATP500,Outdoor,Clay,35.0,136.0,1.75,2.08,196.0,Right-Handed,1284,178.0,Right-Handed,559.0,NaN
7,Barcelona,ATP500,Outdoor,Clay,111.0,143.0,1.45,2.75,193.0,Right-Handed,776,188.0,Right-Handed,659.0,NaN
8,Barcelona,ATP500,Outdoor,Clay,77.0,81.0,3.28,1.34,183.0,Left-Handed,1169,183.0,Right-Handed,1198.0,NaN
9,Barcelona,ATP500,Outdoor,Clay,52.0,89.0,2.25,1.65,188.0,Left-Handed,1247,185.0,Right-Handed,287.0,NaN


In [875]:
def Validation_Labelizer(df):
    dict_idx=0
    for idx,column in enumerate(df):
        
        if df[column].dtype==object:
            df.iloc[:,idx]=df.iloc[:,idx].map(label_dictionnary[dict_idx])
            dict_idx+=1
    return df

In [876]:
df_val_test=Validation_Labelizer(df_val_test)
df_val_test.head(10)

,Location,Series,Court,Surface,ActualRankingP1,ActualRankingP2,AvgP1,AvgP2,HeightP1,HandednessP1,IdP1,HeightP2,HandednessP2,IdP2,P1Winner
0,9,2,1,1,53.0,36.0,1.43,2.85,188.0,1,306,185.0,1,126.0,NaN
1,9,2,1,1,68.0,80.0,2.53,1.52,183.0,1,1563,180.0,1,62.0,NaN
2,9,2,1,1,72.0,78.0,2.83,1.43,188.0,1,1740,196.0,1,594.0,NaN
3,9,2,1,1,146.0,481.0,1.58,2.38,183.0,1,1913,185.0,1,964.0,NaN
4,9,2,1,1,135.0,70.0,2.00,1.81,178.0,1,1213,188.0,1,725.0,NaN
5,9,2,1,1,39.0,48.0,1.23,4.18,178.0,1,1241,183.0,0,1304.0,NaN
6,9,2,1,1,35.0,136.0,1.75,2.08,196.0,1,1284,178.0,1,559.0,NaN
7,9,2,1,1,111.0,143.0,1.45,2.75,193.0,1,776,188.0,1,659.0,NaN
8,9,2,1,1,77.0,81.0,3.28,1.34,183.0,0,1169,183.0,1,1198.0,NaN
9,9,2,1,1,52.0,89.0,2.25,1.65,188.0,0,1247,185.0,1,287.0,NaN


In [877]:
X_validation=df_val_test.drop(['P1Winner'],axis=1)
#y_validation=df_val_test['P1Winner']
df_val_test["Prediction"]=Ml_model.predict(X_validation)

In [878]:
df_val_test.head(10)

,Location,Series,Court,Surface,ActualRankingP1,ActualRankingP2,AvgP1,AvgP2,HeightP1,HandednessP1,IdP1,HeightP2,HandednessP2,IdP2,P1Winner,Prediction
0,9,2,1,1,53.0,36.0,1.43,2.85,188.0,1,306,185.0,1,126.0,NaN,0.642948
1,9,2,1,1,68.0,80.0,2.53,1.52,183.0,1,1563,180.0,1,62.0,NaN,0.412880
2,9,2,1,1,72.0,78.0,2.83,1.43,188.0,1,1740,196.0,1,594.0,NaN,0.375974
3,9,2,1,1,146.0,481.0,1.58,2.38,183.0,1,1913,185.0,1,964.0,NaN,0.691133
4,9,2,1,1,135.0,70.0,2.00,1.81,178.0,1,1213,188.0,1,725.0,NaN,0.479403
5,9,2,1,1,39.0,48.0,1.23,4.18,178.0,1,1241,183.0,0,1304.0,NaN,0.789808
6,9,2,1,1,35.0,136.0,1.75,2.08,196.0,1,1284,178.0,1,559.0,NaN,0.584306
7,9,2,1,1,111.0,143.0,1.45,2.75,193.0,1,776,188.0,1,659.0,NaN,0.657110
8,9,2,1,1,77.0,81.0,3.28,1.34,183.0,0,1169,183.0,1,1198.0,NaN,0.331504
9,9,2,1,1,52.0,89.0,2.25,1.65,188.0,0,1247,185.0,1,287.0,NaN,0.460940
